In [1]:
!pip install --upgrade datasets transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 85.2 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 11.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.51.3
    Uninstalling transformers-4.51.3:
      Successfully uninstalled transformers-4.51.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
bigframes 1.42.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvi

In [2]:
!pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 25.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 5.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 85.0 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.9.41
    Uninstalling nvidia-nvjitlink-cu12-12.9.41:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.9.41
  Attempting uninstall: nvidia-curand-cu12
    Found

**OPT 1.3b**

In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from datasets import load_dataset
from tqdm import tqdm
import json



# -----------------------
# 1. Load OPT-1.3B Model
# -----------------------
model_name = "facebook/opt-1.3b"

# Configure quantization
quantization_config = BitsAndBytesConfig(
    load_in_8bit=True,
    # If 8-bit is still too large, try 4-bit:
    # load_in_4bit=True,
    # bnb_4bit_quant_type="nf4",
    # bnb_4bit_compute_dtype=torch.bfloat16,
)

tokenizer = AutoTokenizer.from_pretrained(model_name)
# Load the model with quantization config
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quantization_config,
)
# When using load_in_8bit or load_in_4bit, the model is automatically moved to the GPU
# So, you don't need the explicit model.to(device) call
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model = model.to(device) # This line is no longer necessary with load_in_8bit/4bit

model.eval()

# -----------------------
# 2. Load Dataset (CommonsenseQA - Reasoning MCQs)
# -----------------------
dataset = load_dataset("commonsense_qa")
test_set = dataset['validation']

# -----------------------
# 3. Prompt Template for Reasoning MCQs
# -----------------------
def generate_prompt(question, choices):
    prompt = f"Question: {question}\nChoices:\n"
    for idx, choice in enumerate(choices):
        prompt += f"{chr(65+idx)}. {choice}\n"
    prompt += "Answer:"
    return prompt

# -----------------------
# 4. Generate Model Output
# -----------------------
def get_model_output(prompt, max_new_tokens=64):
    # Ensure inputs are on the correct device.
    # When using 8-bit or 4-bit, the model is likely on GPU,
    # so inputs should also be on GPU.
    device = model.device # Get the actual device of the model
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=1024).to(device)
    outputs = model.generate(**inputs, max_new_tokens=max_new_tokens)
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return decoded[len(prompt):].strip()

# -----------------------
# 5. Evaluation
# -----------------------
correct = 0
total = 0
predictions = []

print("Running evaluation on CommonsenseQA with OPT-1.3B...\n")

for example in tqdm(test_set, total=len(test_set)):
    question = example['question']
    # Add the check for the structure of 'choices' to handle potential dictionary format
    if isinstance(example['choices'], dict):
        # Assuming the dictionary structure has 'text' and 'label' as lists
        choices_list = [{'text': t, 'label': l} for t, l in zip(example['choices']['text'], example['choices']['label'])]
    else:
        # Otherwise, assume it's already a list of dictionaries
        choices_list = example['choices']

    # Now, safely extract 'text' from the list of dictionaries
    choices = [choice['text'] for choice in choices_list]
    correct_answer = example['answerKey']

    prompt = generate_prompt(question, choices)
    output = get_model_output(prompt)

    is_correct = (correct_answer in output) or (any(opt.lower() in output.lower() for opt in choices if opt == correct_answer))

    predictions.append({
        "question": question,
        "prompt": prompt,
        "output": output,
        "choices": choices,
        "ground_truth_answer": correct_answer,
        "correct": is_correct
    })

    total += 1
    correct += int(is_correct)

accuracy = correct / total if total > 0 else 0.0
print(f"\nFinal Accuracy on CommonsenseQA (Reasoning Task): {accuracy:.4f}")

# -----------------------
# 6. Save Predictions
# -----------------------
with open("opt_commonsenseqa_predictions.json", "w") as f:
    json.dump(predictions, f, indent=2)

print("\nPredictions saved to opt_commonsenseqa_predictions.json")

tokenizer_config.json:   0%|          | 0.00/685 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/653 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/441 [00:00<?, ?B/s]

...

Generating validation split:   0%|          | 0/1221 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1140 [00:00<?, ? examples/s]

Running evaluation on CommonsenseQA with OPT-1.3B...



100%|██████████| 1221/1221 [2:17:27<00:00,  6.75s/it] 


Final Accuracy on CommonsenseQA (Reasoning Task): 0.9558

Predictions saved to opt_commonsenseqa_predictions.json


**STABLE LM 2**

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset
from tqdm import tqdm
import json

# -----------------------
# 1. Load Model (StableLM 2 - 1.6B)
# -----------------------
model_name = "stabilityai/stablelm-2-zephyr-1_6b"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
model.eval()

# -----------------------
# 2. Load Dataset (CommonsenseQA - Reasoning MCQs)
# -----------------------
dataset = load_dataset("commonsense_qa")
test_set = dataset['validation']

# -----------------------
# 3. Prompt Template for Reasoning MCQs
# -----------------------
def generate_prompt(question, choices):
    prompt = f"Question: {question}\nChoices:\n"
    for idx, choice in enumerate(choices):
        prompt += f"{chr(65+idx)}. {choice}\n"
    prompt += "Answer:"
    return prompt

# -----------------------
# 4. Generate Model Output
# -----------------------
def get_model_output(prompt, max_new_tokens=64):
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=1024).to(device)
    outputs = model.generate(**inputs, max_new_tokens=max_new_tokens)
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return decoded[len(prompt):].strip()

# -----------------------
# 5. Evaluation
# -----------------------
correct = 0
total = 0
predictions = []

print("Running evaluation on CommonsenseQA with StableLM 2...\n")

for example in tqdm(test_set, total=len(test_set)):
    question = example['question']
    # Add the check for the structure of 'choices' to handle potential dictionary format
    if isinstance(example['choices'], dict):
        # Assuming the dictionary structure has 'text' and 'label' as lists
        choices_list = [{'text': t, 'label': l} for t, l in zip(example['choices']['text'], example['choices']['label'])]
    else:
        # Otherwise, assume it's already a list of dictionaries
        choices_list = example['choices']

    # Now, safely extract 'text' from the list of dictionaries
    choices = [choice['text'] for choice in choices_list]

    correct_answer = example['answerKey']

    prompt = generate_prompt(question, choices)
    output = get_model_output(prompt)

    is_correct = correct_answer.lower() in output.lower()

    predictions.append({
        "question": question,
        "prompt": prompt,
        "output": output,
        "choices": choices,
        "ground_truth_answer": correct_answer,
        "correct": is_correct
    })

    total += 1
    correct += int(is_correct)

accuracy = correct / total if total > 0 else 0.0
print(f"\nFinal Accuracy on CommonsenseQA (Reasoning Task): {accuracy:.4f}")

# -----------------------
# 6. Save Predictions
# -----------------------
with open("stablelm_commonsenseqa_predictions.json", "w") as f:
    json.dump(predictions, f, indent=2)

print("\nPredictions saved to stablelm_commonsenseqa_predictions.json")

tokenizer_config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.01M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/917k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/784 [00:00<?, ?B/s]

...

Generating validation split:   0%|          | 0/1221 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1140 [00:00<?, ? examples/s]

Running evaluation on CommonsenseQA with StableLM 2...



100%|██████████| 1221/1221 [37:15<00:00,  1.83s/it]


Final Accuracy on CommonsenseQA (Reasoning Task): 0.9943

Predictions saved to stablelm_commonsenseqa_predictions.json


**gpt-neo-1.3B**

In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset
from tqdm import tqdm
import json

# -----------------------
# 1. Load gpt-neo-1.3B Model
# -----------------------
model_name = "EleutherAI/gpt-neo-1.3B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
model.eval()

# -----------------------
# 2. Load Dataset (CommonsenseQA - Reasoning MCQs)
# -----------------------
dataset = load_dataset("commonsense_qa")
test_set = dataset['validation']

# -----------------------
# 3. Prompt Template for Reasoning MCQs
# -----------------------
def generate_prompt(question, choices):
    prompt = f"Question: {question}\nChoices:\n"
    for idx, choice in enumerate(choices):
        prompt += f"{chr(65+idx)}. {choice}\n"
    prompt += "Answer:"
    return prompt

# -----------------------
# 4. Generate Model Output
# -----------------------
def get_model_output(prompt, max_new_tokens=64):
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=1024).to(device)
    outputs = model.generate(**inputs, max_new_tokens=max_new_tokens)
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return decoded[len(prompt):].strip()

# -----------------------
# 5. Evaluation
# -----------------------
correct = 0
total = 0
predictions = []

print("Running evaluation on CommonsenseQA with gpt-neo-1.3B...\n")

for example in tqdm(test_set, total=len(test_set)):
    question = example['question']
    # Add the check for the structure of 'choices' to handle potential dictionary format
    if isinstance(example['choices'], dict):
        # Assuming the dictionary structure has 'text' and 'label' as lists
        choices_list = [{'text': t, 'label': l} for t, l in zip(example['choices']['text'], example['choices']['label'])]
    else:
        # Otherwise, assume it's already a list of dictionaries
        choices_list = example['choices']

    # Now, safely extract 'text' from the list of dictionaries
    choices = [choice['text'] for choice in choices_list]
    correct_answer = example['answerKey']

    prompt = generate_prompt(question, choices)
    output = get_model_output(prompt)

    is_correct = (correct_answer in output) or (any(opt.lower() in output.lower() for opt in choices if opt == correct_answer))

    predictions.append({
        "question": question,
        "prompt": prompt,
        "output": output,
        "choices": choices,
        "ground_truth_answer": correct_answer,
        "correct": is_correct
    })

    total += 1
    correct += int(is_correct)

accuracy = correct / total if total > 0 else 0.0
print(f"\nFinal Accuracy on CommonsenseQA (Reasoning Task): {accuracy:.4f}")

# -----------------------
# 6. Save Predictions
# -----------------------
with open("gptneo_commonsenseqa_predictions.json", "w") as f:
    json.dump(predictions, f, indent=2)

print("\nPredictions saved to gptneo_commonsenseqa_predictions.json")

tokenizer_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/5.31G [00:00<?, ?B/s]

Running evaluation on CommonsenseQA with gpt-neo-1.3B...



100%|██████████| 1221/1221 [39:20<00:00,  1.93s/it]


Final Accuracy on CommonsenseQA (Reasoning Task): 0.9066

Predictions saved to gptneo_commonsenseqa_predictions.json


**falcon-rw-1b**

In [4]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset
from tqdm import tqdm
import json

# -----------------------
# 1. Load Falcon-rw-1b Model
# -----------------------
model_name = "tiiuae/falcon-rw-1b"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
model.eval()

# -----------------------
# 2. Load Dataset (CommonsenseQA - Reasoning MCQs)
# -----------------------
dataset = load_dataset("commonsense_qa")
test_set = dataset['validation']

# -----------------------
# 3. Prompt Template for Reasoning MCQs
# -----------------------
def generate_prompt(question, choices):
    prompt = f"Question: {question}\nChoices:\n"
    for idx, choice in enumerate(choices):
        prompt += f"{chr(65+idx)}. {choice}\n"
    prompt += "Answer:"
    return prompt

# -----------------------
# 4. Generate Model Output
# -----------------------
def get_model_output(prompt, max_new_tokens=64):
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=1024).to(device)
    outputs = model.generate(**inputs, max_new_tokens=max_new_tokens)
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return decoded[len(prompt):].strip()

# -----------------------
# 5. Evaluation
# -----------------------
correct = 0
total = 0
predictions = []

print("Running evaluation on CommonsenseQA with Falcon-rw-1b...\n")

for example in tqdm(test_set, total=len(test_set)):
    question = example['question']
    # Add the check for the structure of 'choices'
    if isinstance(example['choices'], dict):
        # Assuming the dictionary structure has 'text' and 'label' as lists
        choices_list = [{'text': t, 'label': l} for t, l in zip(example['choices']['text'], example['choices']['label'])]
    else:
        # Otherwise, assume it's already a list of dictionaries
        choices_list = example['choices']

    # Now, safely extract 'text' from the list of dictionaries
    choices = [choice['text'] for choice in choices_list]
    correct_answer = example['answerKey']

    prompt = generate_prompt(question, choices)
    output = get_model_output(prompt)

    is_correct = (correct_answer in output) or (any(opt.lower() in output.lower() for opt in choices if opt == correct_answer))

    predictions.append({
        "question": question,
        "prompt": prompt,
        "output": output,
        "choices": choices,
        "ground_truth_answer": correct_answer,
        "correct": is_correct
    })

    total += 1
    correct += int(is_correct)

accuracy = correct / total if total > 0 else 0.0
print(f"\nFinal Accuracy on CommonsenseQA (Reasoning Task): {accuracy:.4f}")

# -----------------------
# 6. Save Predictions
# -----------------------
with open("falcon_commonsenseqa_predictions.json", "w") as f:
    json.dump(predictions, f, indent=2)

print("\nPredictions saved to falcon_commonsenseqa_predictions.json")

tokenizer_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/2.62G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/115 [00:00<?, ?B/s]

Running evaluation on CommonsenseQA with Falcon-rw-1b...




  0%|          | 0/1221 [00:00<?, ?it/s]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.

  0%|          | 1/1221 [00:01<36:28,  1.79s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.

  0%|          | 2/1221 [00:03<34:53,  1.72s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.

  0%|          | 3/1221 [00:05<34:22,  1.69s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.

  0%|          | 4/1221 [00:06<34:14,  1.69s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.

  0%|          | 5/1221 [00:08<34:24,  1.70s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.

  0%|          | 6/1221 [00:10<34:11,  1.69s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.

  1%|          | 7/1221 [00:11<34:06,  1.69s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.

  1%|          | 8/1221 [00:13<33:58,  1.68s/it]Setting `pad_token_id` 


Final Accuracy on CommonsenseQA (Reasoning Task): 0.7477

Predictions saved to falcon_commonsenseqa_predictions.json


**pythia-1b**

In [ ]:

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset
from tqdm import tqdm
import json

# -----------------------
# 1. Load Model
# -----------------------
model_name = "EleutherAI/pythia-1b"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
model.eval()

# -----------------------
# 2. Load Dataset (CommonsenseQA - Reasoning MCQs)
# -----------------------
dataset = load_dataset("commonsense_qa")
test_set = dataset['validation']

# -----------------------
# 3. Prompt Template for Reasoning MCQs
# -----------------------
def generate_prompt(question, choices):
    prompt = f"Question: {question}\nChoices:\n"
    for idx, choice in enumerate(choices):
        prompt += f"{chr(65+idx)}. {choice}\n"
    prompt += "Answer:"
    return prompt

# -----------------------
# 4. Generate Model Output
# -----------------------
def get_model_output(prompt, max_new_tokens=64):
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=1024).to(device)
    outputs = model.generate(**inputs, max_new_tokens=max_new_tokens)
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return decoded[len(prompt):].strip()

# -----------------------
# 5. Evaluation
# -----------------------
correct = 0
total = 0
predictions = []

print("Running evaluation on CommonsenseQA Reasoning Task...\n")

for example in tqdm(test_set, total=len(test_set)):
    question = example['question']
    # Check if 'choices' is a dictionary and convert it to a list if needed
    # Based on dataset inspection, 'choices' can sometimes be a dict with 'text' and 'label' keys
    # We want a list of dictionaries, each containing a 'text' key
    if isinstance(example['choices'], dict):
        # Assuming the dictionary structure has 'text' and 'label' as lists
        choices = [{'text': t, 'label': l} for t, l in zip(example['choices']['text'], example['choices']['label'])]
    else:
        # Otherwise, assume it's already a list of dictionaries
        choices = example['choices']

    # Now, safely extract 'text' from the (now confirmed) list of dictionaries
    choices_text = [choice['text'] for choice in choices]
    correct_answer = example['answerKey']

    prompt = generate_prompt(question, choices_text) # Pass choices_text to generate_prompt
    output = get_model_output(prompt)

    is_correct = (correct_answer in output) or (any(opt.lower() in output.lower() for opt in choices_text if opt == correct_answer)) # Use choices_text here

    predictions.append({
        "question": question,
        "prompt": prompt,
        "output": output,
        "choices": choices_text, # Save choices_text
        "ground_truth_answer": correct_answer,
        "correct": is_correct
    })

    total += 1
    correct += int(is_correct)

accuracy = correct / total if total > 0 else 0.0
print(f"\nFinal Accuracy on CommonsenseQA (Reasoning Task): {accuracy:.4f}")

# -----------------------
# 6. Save Predictions
# -----------------------
with open("commonsenseqa_predictions.json", "w") as f:
    json.dump(predictions, f, indent=2)

print("\nPredictions saved to commonsenseqa_predictions.json")

tokenizer_config.json:   0%|          | 0.00/396 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/569 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.09G [00:00<?, ?B/s]

Running evaluation on CommonsenseQA Reasoning Task...



100%|█████████▉| 1220/1221 [23:57<00:01,  1.18s/it]Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
